In [17]:
import pickle
import pandas as pd
import keras
import mlflow

from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression


import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt


from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np

In [18]:
# Cargar datos MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Aplanar imágenes para modelos clásicos de ML
x_train_flat = x_train.reshape(x_train.shape[0], -1).astype(np.float32) / 255.0
x_test_flat = x_test.reshape(x_test.shape[0], -1).astype(np.float32) / 255.0

# Crear datasets usando MLflow
training_dataset = mlflow.data.from_numpy(x_train_flat, targets=y_train, name="MNIST-dataset-01")
validation_dataset = mlflow.data.from_numpy(x_test_flat, targets=y_test, name="MNIST-dataset-02")


### <font color='darkblue'>dagshub</font> upload experiments

In [ ]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/ioSoyPato/MNIST-experiments", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

In [20]:
# Cargar datos MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Aplanar imágenes para modelos clásicos de ML
x_train_flat = x_train.reshape(x_train.shape[0], -1).astype(np.float32) / 255.0
x_test_flat = x_test.reshape(x_test.shape[0], -1).astype(np.float32) / 255.0

# Crear datasets usando MLflow desde numpy
training_dataset = mlflow.data.from_numpy(x_train_flat, targets=y_train, name="MNIST-dataset-01")
validation_dataset = mlflow.data.from_numpy(x_test_flat, targets=y_test, name="MNIST-dataset-02")


In [ ]:
mlflow.sklearn.autolog()

def objective_rf(params):
    with mlflow.start_run(nested=True):
        # Set model tag
        mlflow.set_tag("model_family", "random_forest")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train RandomForest model
        rf_model = RandomForestRegressor(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            random_state=42
        )
        rf_model.fit(x_train_flat, y_train)
        
        # Predict on validation dataset
        y_pred = rf_model.predict(x_test_flat)
        
        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        # Log RMSE metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

# Define search space for RandomForest
search_space_rf = {
    'n_estimators': hp.quniform('n_estimators', 50, 100, 1),
    'max_depth': hp.quniform('max_depth', 5, 15, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 5, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 2, 1),
}


# Run hyperparameter optimization
with mlflow.start_run(run_name="Parent Random Forest", nested=True):
    best_params_rf = fmin(
        fn=objective_rf,
        space=search_space_rf,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    # Log best parameters
    mlflow.log_params(best_params_rf)
